In [7]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus
import os

# Load CSV files
student_info = pd.read_csv('input/Student_Info.csv')
student_preference_info = pd.read_csv('input/Student_Preference_Info.csv')
teacher_info = pd.read_csv('input/Teacher_Info.csv')
teacher_unavailability = pd.read_csv('input/Teacher_unavailability.csv')
sections_info = pd.read_csv('input/Sections_Information.csv')

# Define periods
periods = ['R1', 'R2', 'R3', 'R4', 'G1', 'G2', 'G3', 'G4']

# Filter sections based on teacher availability and student requests
valid_sections = []
for section_id, teacher_id in zip(sections_info['Section ID'], sections_info['Teacher Assigned']):
    unavailable_periods = teacher_unavailability[teacher_unavailability['Teacher ID'] == teacher_id]['Unavailable Periods'].values
    if len(unavailable_periods) > 0 and pd.notna(unavailable_periods[0]):
        unavailable_periods = unavailable_periods[0].split(',')
    else:
        unavailable_periods = []
    for period in periods:
        if period not in unavailable_periods:
            valid_sections.append((section_id, period))

# Debugging: Print valid sections
print("Valid Sections:")
print(valid_sections)

# Filter sections based on student requests
requested_sections = set()
for student_id in student_info['Student ID']:
    requested_classes = student_preference_info[student_preference_info['Student ID'] == student_id]['Preferred Sections'].values[0].split(',')
    for class_id in requested_classes:
        sections_for_class = sections_info[sections_info['Course ID'] == class_id]['Section ID']
        for section_id in sections_for_class:
            if section_id in [sec_id for sec_id, _ in valid_sections]:
                requested_sections.add(section_id)

# Debugging: Print requested sections
print("Requested Sections:")
print(requested_sections)

# Initialize the problem
prob = LpProblem("School_Scheduling_Problem", LpMaximize)

# Create decision variables
# x[i][j] will be 1 if student i is assigned to section j, 0 otherwise
x = {}
for student_id in student_info['Student ID']:
    for section_id in requested_sections:
        x[(student_id, section_id)] = LpVariable(f"x_{student_id}_{section_id}", 0, 1, cat='Binary')

# z[j][p] will be 1 if section j is scheduled in period p, 0 otherwise
z = {}
for section_id, period in valid_sections:
    z[(section_id, period)] = LpVariable(f"z_{section_id}_{period}", 0, 1, cat='Binary')

# Objective function: Maximize the number of students assigned to their preferred sections
prob += lpSum(x[(student_id, section_id)] for student_id in student_info['Student ID'] for section_id in requested_sections)

# Constraints
# 1. Each student can be assigned to at most one section per period
for student_id in student_info['Student ID']:
    for period in periods:
        prob += lpSum(x[(student_id, section_id)] for section_id in requested_sections if (section_id, period) in valid_sections) <= 1

# 2. Each section must be scheduled in exactly one period
for section_id in sections_info['Section ID']:
    prob += lpSum(z[(section_id, period)] for sec_id, period in valid_sections if sec_id == section_id) == 1

# 3. Link student assignment to section scheduling
for student_id in student_info['Student ID']:
    for section_id in requested_sections:
        prob += x[(student_id, section_id)] <= lpSum(z[(section_id, period)] for sec_id, period in valid_sections if sec_id == section_id)

# 4. Respect section capacities
for section_id in sections_info['Section ID']:
    capacity = sections_info[sections_info['Section ID'] == section_id]['# of Seats Available'].values[0]
    prob += lpSum(x[(student_id, section_id)] for student_id in student_info['Student ID'] if (student_id, section_id) in x) <= capacity

# 5. Ensure each student is assigned to at most one section per requested class
for student_id in student_info['Student ID']:
    requested_classes = student_preference_info[student_preference_info['Student ID'] == student_id]['Preferred Sections'].values[0].split(',')
    for class_id in requested_classes:
        sections_for_class = sections_info[sections_info['Course ID'] == class_id]['Section ID']
        prob += lpSum(x[(student_id, section_id)] for section_id in sections_for_class if section_id in requested_sections) <= 1

# 6. Ensure each student is assigned to all their requested classes
for student_id in student_info['Student ID']:
    requested_classes = student_preference_info[student_preference_info['Student ID'] == student_id]['Preferred Sections'].values[0].split(',')
    prob += lpSum(lpSum(x[(student_id, section_id)] for section_id in sections_info[sections_info['Course ID'] == class_id]['Section ID'] if section_id in requested_sections) for class_id in requested_classes) >= len(requested_classes)

# 7. Ensure Medical Career class has one section in period R1 and one in period G1
medical_career_sections = sections_info[sections_info['Course ID'] == 'Medical Career']['Section ID']
for section_id in medical_career_sections:
    prob += z[(section_id, 'R1')] == 1
    prob += z[(section_id, 'G1')] == 1

# 8. Ensure Heroes Teach class has one section in period R2 and one in period G2
heroes_teach_sections = sections_info[sections_info['Course ID'] == 'Heroes Teach']['Section ID']
for section_id in heroes_teach_sections:
    prob += z[(section_id, 'R2')] == 1
    prob += z[(section_id, 'G2')] == 1

# 9. Ensure Sports Med sections do not overlap
sports_med_sections = sections_info[sections_info['Course ID'] == 'Sports Med']['Section ID']
for period in periods:
    prob += lpSum(z[(section_id, period)] for section_id in sports_med_sections if (section_id, period) in valid_sections) <= 1

# 10. Ensure science department sections have a prep period or lunch if switching class types
science_sections = sections_info[sections_info['Department'] == 'Science']

for section_id1 in science_sections['Section ID']:
    for section_id2 in science_sections['Section ID']:
        if section_id1 != section_id2:
            course_id1 = sections_info[sections_info['Section ID'] == section_id1]['Course ID'].values[0]
            course_id2 = sections_info[sections_info['Section ID'] == section_id2]['Course ID'].values[0]
            if course_id1 != course_id2:
                for period1, period2 in zip(periods[:-1], periods[1:]):
                    if period1 in ['R2', 'G2'] and period2 in ['R3', 'G3']:
                        continue  # Skip lunch periods
                    if (section_id1, period1) in z and (section_id2, period2) in z:
                        prob += z[(section_id1, period1)] + z[(section_id2, period2)] <= 1

# 11. Ensure no more than 12 SPED students per class
sped_students = student_info[student_info['SPED'] == 'Yes']['Student ID']
for section_id in sections_info['Section ID']:
    prob += lpSum(x[(student_id, section_id)] for student_id in sped_students if (student_id, section_id) in x) <= 12

# Solve the problem
prob.solve()

# Output the results
print("Status:", LpStatus[prob.status])

# Create output directory
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# Debugging: Check the values of z variables
print("\nSection Scheduling:")
section_scheduling = []
for section_id, period in valid_sections:
    if z[(section_id, period)].varValue == 1:
        print(f"Section {section_id} is scheduled in period {period}")
        section_scheduling.append({'Section ID': section_id, 'Period': period})

# Output the master schedule
master_schedule_df = pd.DataFrame(section_scheduling)
master_schedule_df.to_csv(os.path.join(output_dir, 'Master_Schedule.csv'), index=False)
print("\nMaster Schedule:")
print(master_schedule_df)

# Debugging: Check the values of x variables
print("\nStudent Assignments:")
student_assignments = []
for student_id in student_info['Student ID']:
    assigned_sections = []
    for section_id in requested_sections:
        if x[(student_id, section_id)].varValue == 1:
            assigned_sections.append(section_id)
            student_assignments.append({'Student ID': student_id, 'Section ID': section_id})
    if assigned_sections:
        print(f"Student {student_id} is assigned to sections: {assigned_sections}")
    else:
        print(f"Student {student_id} is not assigned to any section")

# Output student assignments
student_assignments_df = pd.DataFrame(student_assignments)
student_assignments_df.to_csv(os.path.join(output_dir, 'Student_Assignments.csv'), index=False)

# Output teacher assignments
teacher_assignments = []
for section_id, period in valid_sections:
    if z[(section_id, period)].varValue == 1:
        teacher_id = sections_info[sections_info['Section ID'] == section_id]['Teacher Assigned'].values[0]
        teacher_assignments.append({'Teacher ID': teacher_id, 'Section ID': section_id, 'Period': period})

teacher_assignments_df = pd.DataFrame(teacher_assignments)
teacher_assignments_df.to_csv(os.path.join(output_dir, 'Teacher_Assignments.csv'), index=False)

# Output students who did not get requested classes
students_unmet_requests = []
for student_id in student_info['Student ID']:
    requested_classes = student_preference_info[student_preference_info['Student ID'] == student_id]['Preferred Sections'].values[0].split(',')
    assigned_classes = set()
    for section_id in requested_sections:
        if (student_id, section_id) in x and x[(student_id, section_id)].varValue == 1:
            class_id = sections_info[sections_info['Section ID'] == section_id]['Course ID'].values[0]
            assigned_classes.add(class_id)
    unmet_classes = set(requested_classes) - assigned_classes
    if unmet_classes:
        students_unmet_requests.append({'Student ID': student_id, 'Unmet Requests': ','.join(unmet_classes)})

students_unmet_requests_df = pd.DataFrame(students_unmet_requests)
students_unmet_requests_df.to_csv(os.path.join(output_dir, 'Students_Unmet_Requests.csv'), index=False)
print("\nStudents Without All Requested Courses:")
print(students_unmet_requests_df)

Valid Sections:
[('S001', 'R1'), ('S001', 'R2'), ('S001', 'R3'), ('S001', 'R4'), ('S001', 'G1'), ('S001', 'G2'), ('S001', 'G3'), ('S001', 'G4'), ('S002', 'R1'), ('S002', 'R2'), ('S002', 'R3'), ('S002', 'R4'), ('S002', 'G1'), ('S002', 'G2'), ('S002', 'G3'), ('S002', 'G4'), ('S003', 'R1'), ('S003', 'R2'), ('S003', 'R3'), ('S003', 'R4'), ('S003', 'G1'), ('S003', 'G2'), ('S003', 'G3'), ('S003', 'G4'), ('S004', 'R1'), ('S004', 'R2'), ('S004', 'R3'), ('S004', 'R4'), ('S004', 'G1'), ('S004', 'G2'), ('S004', 'G3'), ('S004', 'G4'), ('S005', 'R1'), ('S005', 'R2'), ('S005', 'R3'), ('S005', 'R4'), ('S005', 'G1'), ('S005', 'G2'), ('S005', 'G3'), ('S005', 'G4'), ('S006', 'R1'), ('S006', 'R2'), ('S006', 'R3'), ('S006', 'R4'), ('S006', 'G1'), ('S006', 'G2'), ('S006', 'G3'), ('S006', 'G4'), ('S007', 'R1'), ('S007', 'R2'), ('S007', 'R3'), ('S007', 'R4'), ('S007', 'G1'), ('S007', 'G2'), ('S007', 'G3'), ('S007', 'G4'), ('S008', 'R1'), ('S008', 'R2'), ('S008', 'R3'), ('S008', 'R4'), ('S008', 'G1'), ('S008'

In [17]:
#check what constraints are unmet:

import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus
import os
from collections import Counter

# Load CSV files
student_info = pd.read_csv('input/Student_Info.csv')
student_preference_info = pd.read_csv('input/Student_Preference_Info.csv')
teacher_info = pd.read_csv('input/Teacher_Info.csv')
teacher_unavailability = pd.read_csv('input/Teacher_unavailability.csv')
sections_info = pd.read_csv('input/Sections_Information.csv')

# Define periods
periods = ['R1', 'R2', 'R3', 'R4', 'G1', 'G2', 'G3', 'G4']

# Filter sections based on teacher availability and student requests
valid_sections = []
for section_id, teacher_id in zip(sections_info['Section ID'], sections_info['Teacher Assigned']):
    unavailable_periods = teacher_unavailability[teacher_unavailability['Teacher ID'] == teacher_id]['Unavailable Periods'].values
    if len(unavailable_periods) > 0 and pd.notna(unavailable_periods[0]):
        unavailable_periods = unavailable_periods[0].split(',')
    else:
        unavailable_periods = []
    for period in periods:
        if period not in unavailable_periods:
            valid_sections.append((section_id, period))

# Debugging: Print valid sections
print("Valid Sections:")
print(valid_sections)

# Create a mapping from courses to their sections
course_to_sections = {}
for _, row in sections_info.iterrows():
    if row['Course ID'] not in course_to_sections:
        course_to_sections[row['Course ID']] = []
    course_to_sections[row['Course ID']].append(row['Section ID'])

# Print mapping for debugging
print("\nCourse to Section Mapping:")
for course, sections in course_to_sections.items():
    print(f"{course}: {sections}")

# Filter sections based on student requests
requested_sections = set()
for student_id in student_info['Student ID']:
    requested_courses = student_preference_info[
        student_preference_info['Student ID'] == student_id
    ]['Preferred Sections'].values[0].split(';')
    
    # For each requested course, add all its sections to the set
    for course_id in requested_courses:
        if course_id in course_to_sections:
            requested_sections.update(course_to_sections[course_id])

print("\nRequested Sections after mapping:")
print(requested_sections)

# Initialize the problem with diagnostics
prob = LpProblem("School_Scheduling_Diagnostic", LpMaximize)

# Create decision variables
x = {}  # student-section assignments
z = {}  # section-period assignments

for student_id in student_info['Student ID']:
    for section_id in requested_sections:
        x[(student_id, section_id)] = LpVariable(f"x_{student_id}_{section_id}", 0, 1, cat='Binary')

for section_id, period in valid_sections:
    z[(section_id, period)] = LpVariable(f"z_{section_id}_{period}", 0, 1, cat='Binary')

# Create diagnostic variables
diagnostic_vars = {
    'missing_courses': {},      # Track students missing required courses
    'section_overload': {},     # Track overcrowded sections
    'period_conflicts': {},     # Track scheduling conflicts
    'sped_overload': {},       # Track SPED distribution issues
    'science_prep_violation': {},  # Track science prep period violations
    'medical_career_violation': {}, # Track Medical Career constraints
    'heroes_teach_violation': {},   # Track Heroes Teach constraints
    'sports_med_violation': {}      # Track Sports Med constraints
}

# Initialize base objective (maximizing assignments)
objective = lpSum(x[(student_id, section_id)] 
                 for student_id in student_info['Student ID'] 
                 for section_id in requested_sections)

# Add constraints with diagnostics

# 1. Each student can be assigned to at most one section per period
for student_id in student_info['Student ID']:
    for period in periods:
        conflict = LpVariable(f"period_conflict_{student_id}_{period}", 0, None)
        diagnostic_vars['period_conflicts'][(student_id, period)] = conflict
        prob += lpSum(x[(student_id, section_id)] 
                     for section_id in requested_sections 
                     if (section_id, period) in valid_sections) <= 1 + conflict
        objective += -750 * conflict

# 2. Each section must be scheduled in exactly one period
for section_id in sections_info['Section ID']:
    prob += lpSum(z[(section_id, period)] 
                 for sec_id, period in valid_sections if sec_id == section_id) == 1

# 3. Link student assignment to section scheduling
for student_id in student_info['Student ID']:
    for section_id in requested_sections:
        prob += x[(student_id, section_id)] <= lpSum(z[(section_id, period)] 
                for sec_id, period in valid_sections if sec_id == section_id)

# 4. Section capacity constraints with diagnostics
for section_id in sections_info['Section ID']:
    overload = LpVariable(f"section_overload_{section_id}", 0, None)
    diagnostic_vars['section_overload'][section_id] = overload
    
    capacity = sections_info[sections_info['Section ID'] == section_id]['# of Seats Available'].values[0]
    prob += lpSum(x[(student_id, section_id)] 
                 for student_id in student_info['Student ID'] 
                 if (student_id, section_id) in x) <= capacity + overload
    objective += -500 * overload

# 5. One section per requested course constraint
for student_id in student_info['Student ID']:
    requested_courses = student_preference_info[
        student_preference_info['Student ID'] == student_id
    ]['Preferred Sections'].values[0].split(';')
    
    for course_id in requested_courses:
        course_sections = course_to_sections.get(course_id, [])
        # Student can take at most one section of each course
        prob += lpSum(x[(student_id, section_id)] 
                     for section_id in course_sections
                     if (student_id, section_id) in x) <= 1

# 6. Required courses constraint with diagnostics
for student_id in student_info['Student ID']:
    requested_courses = student_preference_info[
        student_preference_info['Student ID'] == student_id
    ]['Preferred Sections'].values[0].split(';')
    
    for course_id in requested_courses:
        slack = LpVariable(f"missing_courses_{student_id}_{course_id}", 0, 1)
        diagnostic_vars['missing_courses'][(student_id, course_id)] = slack
        
        if course_id in course_to_sections:
            course_sections = course_to_sections[course_id]
            # Must get exactly one section of each course
            prob += (lpSum(x[(student_id, section_id)] 
                    for section_id in course_sections 
                    if (student_id, section_id) in x) + slack == 1)
            
            objective += -1000 * slack  # High penalty for missing a required course

# 7. Medical Career constraints with diagnostics
medical_career_sections = sections_info[sections_info['Course ID'] == 'Medical Career']['Section ID']
for section_id in medical_career_sections:
    r1_violation = LpVariable(f"medical_r1_{section_id}", 0, None)
    g1_violation = LpVariable(f"medical_g1_{section_id}", 0, None)
    diagnostic_vars['medical_career_violation'][section_id] = (r1_violation, g1_violation)
    
    prob += z[(section_id, 'R1')] + r1_violation == 1
    prob += z[(section_id, 'G1')] + g1_violation == 1
    objective += -800 * (r1_violation + g1_violation)

# 8. Heroes Teach constraints with diagnostics
heroes_teach_sections = sections_info[sections_info['Course ID'] == 'Heroes Teach']['Section ID']
for section_id in heroes_teach_sections:
    r2_violation = LpVariable(f"heroes_r2_{section_id}", 0, None)
    g2_violation = LpVariable(f"heroes_g2_{section_id}", 0, None)
    diagnostic_vars['heroes_teach_violation'][section_id] = (r2_violation, g2_violation)
    
    prob += z[(section_id, 'R2')] + r2_violation == 1
    prob += z[(section_id, 'G2')] + g2_violation == 1
    objective += -800 * (r2_violation + g2_violation)

# 9. Sports Med constraints with diagnostics
sports_med_sections = sections_info[sections_info['Course ID'] == 'Sports Med']['Section ID']
for period in periods:
    overlap = LpVariable(f"sports_med_overlap_{period}", 0, None)
    diagnostic_vars['sports_med_violation'][period] = overlap
    prob += lpSum(z[(section_id, period)] 
                 for section_id in sports_med_sections 
                 if (section_id, period) in valid_sections) <= 1 + overlap
    objective += -600 * overlap

# 10. Science prep period constraints with diagnostics
science_sections = sections_info[sections_info['Department'] == 'Science']
for section_id1 in science_sections['Section ID']:
    for section_id2 in science_sections['Section ID']:
        if section_id1 != section_id2:
            course_id1 = sections_info[sections_info['Section ID'] == section_id1]['Course ID'].values[0]
            course_id2 = sections_info[sections_info['Section ID'] == section_id2]['Course ID'].values[0]
            if course_id1 != course_id2:
                for period1, period2 in zip(periods[:-1], periods[1:]):
                    if period1 in ['R2', 'G2'] and period2 in ['R3', 'G3']:
                        continue
                    if (section_id1, period1) in z and (section_id2, period2) in z:
                        violation = LpVariable(f"science_prep_{section_id1}_{section_id2}_{period1}_{period2}", 0, None)
                        diagnostic_vars['science_prep_violation'][(section_id1, section_id2, period1, period2)] = violation
                        prob += z[(section_id1, period1)] + z[(section_id2, period2)] <= 1 + violation
                        objective += -400 * violation

# 11. SPED distribution constraints with diagnostics
sped_students = student_info[student_info['SPED'] == 'Yes']['Student ID']
for section_id in sections_info['Section ID']:
    sped_overload = LpVariable(f"sped_overload_{section_id}", 0, None)
    diagnostic_vars['sped_overload'][section_id] = sped_overload
    prob += lpSum(x[(student_id, section_id)] 
                 for student_id in sped_students 
                 if (student_id, section_id) in x) <= 12 + sped_overload
    objective += -250 * sped_overload

# Set the objective
prob += objective

# Solve the problem
prob.solve()

def analyze_conflicts():
    """Analyze and report all constraint violations"""
    conflicts = {
        'missing_courses': [],
        'overloaded_sections': [],
        'period_conflicts': [],
        'sped_issues': [],
        'science_prep_issues': [],
        'special_course_issues': []
    }
    
    print("\n=== Scheduling Conflict Analysis ===")
    print(f"\nSolver Status: {LpStatus[prob.status]}")
    
    # Check missing courses - now checking per course
    missing_courses_found = False
    for (student_id, course_id), var in diagnostic_vars['missing_courses'].items():
        if var.varValue > 0:
            missing_courses_found = True
            conflicts['missing_courses'].append({
                'student': student_id,
                'course': course_id,
                'missing_count': 1
            })
    
    if missing_courses_found:
        print("\nStudents Missing Required Courses:")
        student_courses = {}
        for conflict in conflicts['missing_courses']:
            if conflict['student'] not in student_courses:
                student_courses[conflict['student']] = []
            student_courses[conflict['student']].append(conflict['course'])
            
        for student_id, missing_courses in student_courses.items():
            print(f"- Student {student_id} is missing {len(missing_courses)} courses:")
            for course in missing_courses:
                print(f"    - {course}")
    
    # Check section overloads
    overloads_found = False
    for section_id, var in diagnostic_vars['section_overload'].items():
        if var.varValue > 0:
            overloads_found = True
            course_id = sections_info[sections_info['Section ID'] == section_id]['Course ID'].values[0]
            conflicts['overloaded_sections'].append({
                'section': section_id,
                'course': course_id,
                'overload': int(var.varValue)
            })
    
    if overloads_found:
        print("\nOverloaded Sections:")
        for conflict in conflicts['overloaded_sections']:
            print(f"- Section {conflict['section']} ({conflict['course']}) is overloaded by {conflict['overload']} students")
    
    # Check special course constraints
    special_issues_found = False
    
    # Medical Career
    for section_id, (r1_var, g1_var) in diagnostic_vars['medical_career_violation'].items():
        if r1_var.varValue > 0 or g1_var.varValue > 0:
            special_issues_found = True
            conflicts['special_course_issues'].append({
                'type': 'Medical Career',
                'section': section_id,
                'issue': 'Not scheduled in required periods (R1/G1)'
            })
    
    # Heroes Teach
    for section_id, (r2_var, g2_var) in diagnostic_vars['heroes_teach_violation'].items():
        if r2_var.varValue > 0 or g2_var.varValue > 0:
            special_issues_found = True
            conflicts['special_course_issues'].append({
                'type': 'Heroes Teach',
                'section': section_id,
                'issue': 'Not scheduled in required periods (R2/G2)'
            })
    
    # Sports Med
    for period, var in diagnostic_vars['sports_med_violation'].items():
        if var.varValue > 0:
            special_issues_found = True
            conflicts['special_course_issues'].append({
                'type': 'Sports Med',
                'period': period,
                'issue': 'Sections overlap in same period'
            })
    
    if special_issues_found:
        print("\nSpecial Course Scheduling Issues:")
        for conflict in conflicts['special_course_issues']:
            print(f"- {conflict['type']}: {conflict['issue']}")
    
    # Generate recommendations
    print("\n=== Recommendations ===")
    
    if conflicts['missing_courses']:
        print("\nTo resolve missing courses:")
        course_counts = Counter([
            conflict['course']
            for conflict in conflicts['missing_courses']
        ])
        for course, count in course_counts.most_common():
            print(f"- Add capacity for {count} more students in {course}")
    
    if conflicts['overloaded_sections']:
        print("\nTo resolve section overloads:")
        for conflict in conflicts['overloaded_sections']:
            print(f"- Increase capacity of {conflict['course']} section {conflict['section']} by {conflict['overload']} seats")
            print(f"  OR add new section to accommodate {conflict['overload']} students")
    
    if conflicts['special_course_issues']:
        print("\nTo resolve special course issues:")
        for conflict in conflicts['special_course_issues']:
            if conflict['type'] == 'Medical Career':
                print(f"- Ensure Medical Career section {conflict['section']} is scheduled in both R1 and G1")
            elif conflict['type'] == 'Heroes Teach':
                print(f"- Ensure Heroes Teach section {conflict['section']} is scheduled in both R2 and G2")
            elif conflict['type'] == 'Sports Med':
                print(f"- Resolve Sports Med sections overlap in period {conflict['period']}")

    return conflicts

# Output results and create CSV files
# Create output directory
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# Output section scheduling
print("\nSection Scheduling:")
section_scheduling = []
for section_id, period in valid_sections:
    if z[(section_id, period)].varValue == 1:
        print(f"Section {section_id} is scheduled in period {period}")
        section_scheduling.append({'Section ID': section_id, 'Period': period})

# Output the master schedule
master_schedule_df = pd.DataFrame(section_scheduling)
master_schedule_df.to_csv(os.path.join(output_dir, 'Master_Schedule.csv'), index=False)
print("\nMaster Schedule:")
print(master_schedule_df)

# Output student assignments
print("\nStudent Assignments:")
student_assignments = []
for student_id in student_info['Student ID']:
    assigned_sections = []
    for section_id in requested_sections:
        if (student_id, section_id) in x and x[(student_id, section_id)].varValue == 1:
            assigned_sections.append(section_id)
            student_assignments.append({'Student ID': student_id, 'Section ID': section_id})
    if assigned_sections:
        print(f"Student {student_id} is assigned to sections: {assigned_sections}")
    else:
        print(f"Student {student_id} is not assigned to any section")

# Create student assignments CSV
student_assignments_df = pd.DataFrame(student_assignments)
student_assignments_df.to_csv(os.path.join(output_dir, 'Student_Assignments.csv'), index=False)

# Output teacher assignments
teacher_assignments = []
for section_id, period in valid_sections:
    if z[(section_id, period)].varValue == 1:
        teacher_id = sections_info[sections_info['Section ID'] == section_id]['Teacher Assigned'].values[0]
        teacher_assignments.append({
            'Teacher ID': teacher_id,
            'Section ID': section_id,
            'Period': period
        })

# Create teacher assignments CSV
teacher_assignments_df = pd.DataFrame(teacher_assignments)
teacher_assignments_df.to_csv(os.path.join(output_dir, 'Teacher_Assignments.csv'), index=False)

# Output unmet requests
students_unmet_requests = []
for student_id in student_info['Student ID']:
    requested_courses = student_preference_info[
        student_preference_info['Student ID'] == student_id
    ]['Preferred Sections'].values[0].split(';')
    
    assigned_courses = set()
    for section_id in requested_sections:
        if (student_id, section_id) in x and x[(student_id, section_id)].varValue == 1:
            course_id = sections_info[
                sections_info['Section ID'] == section_id
            ]['Course ID'].values[0]
            assigned_courses.add(course_id)
    
    unmet_courses = set(requested_courses) - assigned_courses
    if unmet_courses:
        students_unmet_requests.append({
            'Student ID': student_id,
            'Unmet Requests': ','.join(unmet_courses)
        })

# Create unmet requests CSV
students_unmet_requests_df = pd.DataFrame(students_unmet_requests)
students_unmet_requests_df.to_csv(os.path.join(output_dir, 'Students_Unmet_Requests.csv'), index=False)
print("\nStudents Without All Requested Courses:")
print(students_unmet_requests_df)

# Run the conflict analysis
analyze_conflicts()

Valid Sections:
[('S001', 'R1'), ('S001', 'R2'), ('S001', 'R3'), ('S001', 'R4'), ('S001', 'G1'), ('S001', 'G2'), ('S001', 'G3'), ('S001', 'G4'), ('S002', 'R1'), ('S002', 'R2'), ('S002', 'R3'), ('S002', 'R4'), ('S002', 'G1'), ('S002', 'G2'), ('S002', 'G3'), ('S002', 'G4'), ('S003', 'R1'), ('S003', 'R2'), ('S003', 'R3'), ('S003', 'R4'), ('S003', 'G1'), ('S003', 'G2'), ('S003', 'G3'), ('S003', 'G4'), ('S004', 'R1'), ('S004', 'R2'), ('S004', 'R3'), ('S004', 'R4'), ('S004', 'G1'), ('S004', 'G2'), ('S004', 'G3'), ('S004', 'G4'), ('S005', 'R1'), ('S005', 'R2'), ('S005', 'R3'), ('S005', 'R4'), ('S005', 'G1'), ('S005', 'G2'), ('S005', 'G3'), ('S005', 'G4'), ('S006', 'R1'), ('S006', 'R2'), ('S006', 'R3'), ('S006', 'R4'), ('S006', 'G1'), ('S006', 'G2'), ('S006', 'G3'), ('S006', 'G4'), ('S007', 'R1'), ('S007', 'R2'), ('S007', 'R3'), ('S007', 'R4'), ('S007', 'G1'), ('S007', 'G2'), ('S007', 'G3'), ('S007', 'G4'), ('S008', 'R1'), ('S008', 'R2'), ('S008', 'R3'), ('S008', 'R4'), ('S008', 'G1'), ('S008'

{'missing_courses': [{'student': 'ST001',
   'course': 'English 10-P',
   'missing_count': 1},
  {'student': 'ST001', 'course': 'Integ Math 2-P', 'missing_count': 1},
  {'student': 'ST001', 'course': 'World History-P', 'missing_count': 1},
  {'student': 'ST001', 'course': 'PE', 'missing_count': 1},
  {'student': 'ST001', 'course': 'Spanish 2-P', 'missing_count': 1},
  {'student': 'ST002', 'course': 'English 10-P', 'missing_count': 1},
  {'student': 'ST002', 'course': 'Chem-Earth Syst', 'missing_count': 1},
  {'student': 'ST002', 'course': 'World History-P', 'missing_count': 1},
  {'student': 'ST002', 'course': 'PE', 'missing_count': 1},
  {'student': 'ST002', 'course': 'Intro Art-P', 'missing_count': 1},
  {'student': 'ST003', 'course': 'English 10-P', 'missing_count': 1},
  {'student': 'ST003', 'course': 'Integ Math 2-P', 'missing_count': 1},
  {'student': 'ST003', 'course': 'World History-P', 'missing_count': 1},
  {'student': 'ST003', 'course': 'PE', 'missing_count': 1},
  {'student

In [9]:
# Calculate the number of unmet requests per class
unmet_requests_count = {}
for unmet_request in students_unmet_requests:
    unmet_classes = unmet_request['Unmet Requests'].split(',')
    for class_id in unmet_classes:
        if class_id in unmet_requests_count:
            unmet_requests_count[class_id] += 1
        else:
            unmet_requests_count[class_id] = 1

# Calculate the additional seats needed per class
additional_seats_needed = {}
for class_id, count in unmet_requests_count.items():
    additional_seats_needed[class_id] = count

# Print the recommendation message
print("Classes that need more sections or more capacity:")
for class_id, seats_needed in additional_seats_needed.items():
    print(f"Class {class_id} needs {seats_needed} more seats.")

Classes that need more sections or more capacity:
Class S001;S004;S007;S009;S011;S013 needs 3 more seats.
Class S002;S005;S007;S009;S011;S015 needs 2 more seats.
Class S003;S006;S008;S010;S012;S013 needs 2 more seats.
Class S001;S004;S008;S010;S012;S014 needs 1 more seats.
Class S016;S018;S020;S022;S024;S013 needs 3 more seats.
Class S017;S019;S020;S022;S024;S015 needs 1 more seats.
Class S016;S018;S021;S023;S025;S014 needs 1 more seats.
Class S017;S019;S021;S023;S025;S015 needs 3 more seats.
Class S030;S028;S026;S024;S013;S015 needs 3 more seats.
Class S031;S029;S027;S025;S014;S015 needs 4 more seats.
Class S002;S005;S008;S010;S012;S014 needs 2 more seats.
Class S003;S006;S007;S009;S011;S015 needs 1 more seats.
Class S017;S019;S021;S023;S025;S013 needs 1 more seats.
Class S030;S028;S026;S024;S014;S015 needs 1 more seats.
Class S016;S018;S020;S022;S024;S015 needs 1 more seats.
Class S002;S005;S007;S009;S011;S014 needs 1 more seats.
